# Radiomic Feature Post-processing 

In [1]:
# Intention Hassan transform: Render feature independent of gray level discretization.

In [12]:
import sys
sys.path.append('./../utils/')

import os
import re

import numpy as np
import pandas as pd

import ioutil
import transforms

from processing import PostProcessor
from sklearn.preprocessing import StandardScaler

In [13]:
true_index = np.load('./../../../data_source/patient_id.npy')

## Process Raw Features

In [14]:
ct_feature_files = [
    './../../../data_source/radiomic_features/gauss/ct_32_05gauss_config.csv',
    './../../../data_source/radiomic_features/gauss/ct_64_05gauss_config.csv',
    './../../../data_source/radiomic_features/gauss/ct_128_05gauss_config.csv',
    #'./../../../data_source/radiomic_features/sqroot/ct_32_sqroot_config.csv',
    #'./../../../data_source/radiomic_features/sqroot/ct_64_sqroot_config.csv',
    #'./../../../data_source/radiomic_features/sqroot/ct_128_sqroot_config.csv'
]
pet_feature_files = [
    './../../../data_source/radiomic_features/gauss/pet_32_05gauss_config.csv',
    './../../../data_source/radiomic_features/gauss/pet_64_05gauss_config.csv',
    './../../../data_source/radiomic_features/gauss/pet_128_05gauss_config.csv',
    #'./../../../data_source/radiomic_features/sqroot/pet_32_sqroot_config.csv',
    #'./../../../data_source/radiomic_features/sqroot/pet_64_sqroot_config.csv',
    #'./../../../data_source/radiomic_features/sqroot/pet_128_sqroot_config.csv'
]

In [15]:
ct_processor = PostProcessor(
    path_to_features=ct_feature_files, 
    indices=true_index, 
    filter_type='sigma', 
    error_dir='./../../../data_source/to_analysis/gauss/errors/'
)
ct_processor.produce(drop_redundant=True, drop_missing=True)
ct_processor.to_file(
    path_to_dir='./../../../data_source/to_analysis/gauss/'
)

In [16]:
pet_processor = PostProcessor(
    path_to_features=pet_feature_files, 
    indices=true_index, 
    filter_type='sigma', 
    error_dir='./../../../data_source/to_analysis/gauss/errors/'
)
pet_processor.produce(drop_redundant=True, drop_missing=True)
pet_processor.to_file(
    path_to_dir='./../../../data_source/to_analysis/gauss//'
)

In [17]:
ct_features = ct_processor.concatenated
pet_features = pet_processor.concatenated
np.shape(ct_features), np.shape(pet_features)

((198, 276), (198, 276))

## Concatenate Features

In [18]:
shape_feats = pd.read_csv('./../../../data_source/to_analysis/radiomics_shape.csv', index_col=0)
clinical = pd.read_csv('./../../../data_source/to_analysis/clinical_params.csv', index_col=0)
pet_params = pd.read_csv('./../../../data_source/to_analysis/pet_params.csv', index_col=0)

In [19]:
ct_features.columns = ['CT_{}'.format(col) for col in ct_features.columns]
pet_features.columns = ['PET_{}'.format(col) for col in pet_features.columns]

In [20]:
X = pd.concat((shape_feats, clinical, ct_features, pet_features, pet_params), axis=1)
np.shape(X)

(198, 610)

In [21]:
X.to_csv('./../../../data_source/to_analysis/gauss05_concat.csv')